In [1]:
import matplotlib.pyplot as plt
import numpy as np

from commpy.modulation import QAMModem

from optic.dsp import pulseShape, firFilter, decimate, symbolSync, resample
from optic.models import phaseNoise, linFiberCh, KramersKronigRx, photodiode, awgn

from optic.tx import simpleWDMTx
from optic.core import parameters
from optic.equalization import edc, mimoAdaptEqualizer
from optic.carrierRecovery import cpr
from optic.metrics import fastBERcalc, monteCarloGMI, monteCarloMI, signal_power
from optic.plot import pconst

import scipy.constants as const
from tqdm.notebook import tqdm

from tensorflow.keras.layers import Dense, BatchNormalization, Conv1DTranspose, Conv1D, Flatten, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
figurePath = 'C:/Users/optic/Documents/PIVIC-PIBIC-Comunicacoes-Opticas/resultados/Figuras/Pibic'
path_data  = r'C:/Users/optic/Documents/PIVIC-PIBIC-Comunicacoes-Opticas/resultados/Data/Pibic/'
path_mlp   = 'C:/Users/optic/Documents/PIVIC-PIBIC-Comunicacoes-Opticas/models/NN_models/' 
path_conv  = 'C:/Users/optic/Documents/PIVIC-PIBIC-Comunicacoes-Opticas/models/CONV_models/'

saveData = True
N = 128 # number of input amplitude samples to the NN
Interactions = 5

## Simulation of a single polarization optical signal transmission

In [3]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 400000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
#paramTx.alphaRRC = 0.01       # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 100     # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
FO = paramTx.Rs/2 + 1e9  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12       # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate
Ts = 1 / Fs

# photodiode parameters
paramPD = parameters()
paramPD.B  = 1.1*paramTx.Rs
paramPD.Fs = Fs

## Run BER vs rolloff vs FO for all cases

In [5]:
Rolloff = np.arange(0.05, 0.95, 0.05)
FO_Values = np.arange(0e9, 6e9, 1e9)

BER = np.zeros((3, len(Rolloff), len(FO_Values)))
SER = np.zeros((3, len(Rolloff), len(FO_Values))) 
GMI = np.zeros((3, len(Rolloff), len(FO_Values)))
MI  = np.zeros((3, len(Rolloff), len(FO_Values)))
SNR = np.zeros((3, len(Rolloff), len(FO_Values)))
SIR = np.zeros((3, len(Rolloff), len(FO_Values)))

for indfile in range(0, Interactions):
    for indAlg, alg in enumerate(['KK', 'MLP', 'CONVNET']):
        for indFO, FOfreq in enumerate(tqdm(FO_Values)):
            for indRolloff, rollOff in enumerate(tqdm(Rolloff)):
                
                paramTx.alphaRRC = rollOff
                FO = paramTx.Rs/2 + FOfreq

                # generate optical signal signal
                sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

                # simulate linear signal propagation
                sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

                symbTx = symbTx_[:, :, chIndex]
                Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

                # generate LO field
                π = np.pi
                t = np.arange(0, len(sigCh))*Ts
                ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

                sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

                # Add LO to the received signal
                sigRx = np.sqrt(Plo) + sigCh* np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
                sfm   = sigRx.copy()

                print('CSPR = %.2f dB'%(10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

                # simulate ideal direct-detection optical receiver
                Ipd = photodiode(sigRx, paramPD)
                Amp = np.sqrt(Ipd.real)
                Amp = resample(Amp, paramADC).real

                # resampling to ADC sampling rate
                sigCh = resample(sigCh, paramADC)
                sfm = resample(sfm, paramADC)
                newFs = paramADC.SpS_out*paramTx.Rs

                sfm = sfm/np.sqrt(signal_power(sfm))

                if alg == 'KK':
                    # Kramers-Kronig phase-retrieval
                    phiTime = KramersKronigRx(Amp, newFs)
                    # optical field reconstruction
                    sigRx = Amp*np.exp(1j*phiTime)
                
                elif alg == 'MLP':
                    # Mlp phase-retrieval
                    model = tf.keras.models.load_model(path_mlp+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2)))
                    #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                    sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                    # create set of input features
                    X_input = np.zeros((len(sfm), N)) #(L,N)

                    for indPhase in range(len(sfm)):
                        X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                    sigRx_NN = model.predict(X_input)

                    # optical field reconstruction
                    sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]

                elif alg == 'CONVNET':
                    # ConvNet phase-retrieval
                    model = tf.keras.models.load_model(path_conv+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2)))
                    #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                    sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)

                    # create set of input features
                    X_input = np.zeros((len(sfm), N)) #(L,N)

                    for indPhase in range(len(sfm)):
                        X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                    sigRx_NN = model.predict(X_input)

                    # optical field reconstruction
                    sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]
                    
                # remove DC level
                sigRx -= np.mean(sigRx) # np.sqrt(Plo)
                
                # downshift to baseband
                t = np.arange(0, len(sigRx))*(1/newFs)
                sigRx *= np.exp(-1j * (2 * π * FO * t))

                # Matched filtering
                if paramTx.pulse == "nrz":
                    pulse = pulseShape("nrz", paramADC.SpS_out)
                elif paramTx.pulse == "rrc":
                    pulse = pulseShape(
                        "rrc", paramADC.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs
                    )

                pulse = pulse / np.max(np.abs(pulse))
                sigRx = firFilter(pulse, sigRx)
                sigCh = firFilter(pulse, sigCh)

                # correct for (possible) phase ambiguity
                rot = np.mean(sigCh/sigRx)
                sigRx = rot * sigRx
                sigRx = sigRx / np.sqrt(signal_power(sigRx))

                intf = sigRx/np.sqrt(signal_power(sigRx))-sigCh/np.sqrt(signal_power(sigCh))

                SIR[indAlg, indRolloff, indFO] = 1/signal_power(intf)

                # resample to 2 samples/symbol:
                paramRes = parameters()
                paramRes.Rs = paramTx.Rs
                paramRes.SpS_in  = paramADC.SpS_out
                paramRes.SpS_out = 2

                sigRx = resample(sigRx, paramRes)

                # CD compensation
                sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

                # Downsampling to 2 sps and re-synchronization with transmitted sequences
                sigRx = sigRx.reshape(-1, 1)

                symbRx = symbolSync(sigRx, symbTx, 2)

                # Power normalization
                x = sigRx
                d = symbRx

                x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
                d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

                # Adaptive equalization          
                paramEq = parameters()
                paramEq.nTaps = 15
                paramEq.SpS = 2
                paramEq.mu = [1e-3, 5e-4]
                paramEq.numIter = 5
                paramEq.storeCoeff = False
                paramEq.alg = ["da-rde", "rde"]
                paramEq.M = paramTx.M
                paramEq.L = [20000, 80000]
                paramEq.prgsBar = False

                y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

                # Carrier phase recovery
                paramCPR = parameters()
                paramCPR.alg = "bps"
                paramCPR.M = paramTx.M
                paramCPR.N = 85
                paramCPR.B = 64
                paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

                y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

                y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

                # correct for (possible) phase ambiguity
                for k in range(y_CPR.shape[1]):
                    rot = np.mean(d[:, k] / y_CPR[:, k])
                    y_CPR[:, k] = rot * y_CPR[:, k]

                y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

                discard = int(paramEq.L[0]/2)

                ind = np.arange(discard, d.shape[0] - discard)
                BER[indAlg,indRolloff,indFO], SER[indAlg,indRolloff,indFO], SNR[indAlg,indRolloff,indFO] = fastBERcalc(y_CPR[ind, :], d[ind, :], paramTx.M, 'qam')
                GMI[indAlg,indRolloff,indFO], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')
                MI[indAlg,indRolloff,indFO] = monteCarloMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')

                print("Results:")
                print("BER: %.2e" %(BER[indAlg,indRolloff,indFO]))
                print("SNR: %.2f dB" %(SNR[indAlg,indRolloff,indFO]))
                print('SIR = ', round(10*np.log10(SIR[indAlg,indRolloff,indFO]), 2), ' dB')
                print("GMI: %.2f bits\n" %(GMI[indAlg,indRolloff,indFO]))

    if saveData:    
        # save the simulation data
        np.save(path_data+'BER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), BER)
        np.save(path_data+'SNR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SNR)
        np.save(path_data+'SIR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SIR)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB


KeyboardInterrupt: 

## Simulation of a single polarization optical signal transmission

In [6]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 400000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
paramTx.alphaRRC = 0.5         # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 100     # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
FO = paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
#Plo_dBm = 12      # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate
Ts = 1 / Fs

# photodiode parameters
paramPD = parameters()
paramPD.B  = 1.1*paramTx.Rs
paramPD.Fs = Fs

## Run all CSPR variations

In [10]:
loPower = np.arange(6,16,1)
SNR_values = np.arange(10,20,1)

BER = np.zeros((3, len(loPower), len(SNR_values)))
SER = np.zeros((3, len(loPower), len(SNR_values)))
GMI = np.zeros((3, len(loPower), len(SNR_values)))
MI  = np.zeros((3, len(loPower), len(SNR_values)))
SNR = np.zeros((3, len(loPower), len(SNR_values)))
SIR = np.zeros((3, len(loPower), len(SNR_values)))

for indfile in range(0, 1):
    for indAlg, alg in enumerate(['KK', 'MLP', 'CONVNET']):
        for indPower, Plo_dBm in enumerate(tqdm(loPower)):
            for indSNR, SNRdB in enumerate(tqdm(SNR_values)):

                # generate optical signal signal
                sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)
                        
                # simulate linear signal propagation
                sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

                symbTx = symbTx_[:, :, chIndex]
                
                # AWGN channel
                symbTx = awgn(symbTx, SNRdB, Fs, paramPD.B)

                Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W
                
                # generate LO field
                π = np.pi
                t = np.arange(0, len(sigCh))*Ts
                ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

                sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

                # Add LO to the received signal
                sigRx = np.sqrt(Plo) + sigCh* np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
                sfm   = sigRx.copy()

                print('CSPR = %.2f dB'%(10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

                # simulate ideal direct-detection optical receiver
                Ipd = photodiode(sigRx, paramPD)
                Amp = np.sqrt(Ipd.real)
                Amp = resample(Amp, paramADC).real

                # resampling to ADC sampling rate
                sigCh = resample(sigCh, paramADC)
                sfm = resample(sfm, paramADC)
                newFs = paramADC.SpS_out*paramTx.Rs

                sfm = sfm/np.sqrt(signal_power(sfm))
                if alg == 'KK':
                    # Kramers-Kronig phase-retrieval
                    phiTime = KramersKronigRx(Amp, newFs)
                    # optical field reconstruction
                    sigRx = Amp*np.exp(1j*phiTime)

                elif alg == 'MLP':
                    # Mlp phase-retrieval
                    model = tf.keras.models.load_model(path_mlp+'testModel_SpS_'+str(paramADC.SpS_out)+'_CSPR_'+str(Plo_dBm)+'dB') 
                    #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                    sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                    # create set of input features
                    X_input = np.zeros((len(sfm), N)) #(L,N)

                    for indPhase in range(len(sfm)):
                        X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                    sigRx_NN = model.predict(X_input)
                    # optical field reconstruction
                    sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]

                elif alg == 'CONVNET':
                    # ConvNet phase-retrieval
                    model = tf.keras.models.load_model(path_conv+'testModel_SpS_'+str(paramADC.SpS_out)+'_CSPR_'+str(Plo_dBm)+'dB')
                    #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                    sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                    # create set of input features
                    X_input = np.zeros((len(sfm), N)) #(L,N)

                    for indPhase in range(len(sfm)):
                        X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                    sigRx_NN = model.predict(X_input)
                    # optical field reconstruction
                    sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]    

                # remove DC level
                sigRx -= np.mean(sigRx) # np.sqrt(Plo)  

                # downshift to baseband
                t = np.arange(0, len(sigRx))*(1/newFs)
                sigRx *= np.exp(-1j * (2 * π * FO * t))
                
                # Matched filtering
                if paramTx.pulse == "nrz":
                    pulse = pulseShape("nrz", paramADC.SpS_out)
                elif paramTx.pulse == "rrc":
                    pulse = pulseShape(
                        "rrc", paramADC.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs
                    )

                pulse = pulse / np.max(np.abs(pulse))
                sigRx = firFilter(pulse, sigRx)
                sigCh = firFilter(pulse, sigCh)
                
                # correct for (possible) phase ambiguity
                rot = np.mean(sigCh/sigRx)
                sigRx = rot * sigRx
                sigRx = sigRx / np.sqrt(signal_power(sigRx))

                intf = sigRx/np.sqrt(signal_power(sigRx))-sigCh/np.sqrt(signal_power(sigCh))

                SIR[indAlg, indPower] = 1/signal_power(intf)


                # resample to 2 samples/symbol:
                paramRes = parameters()
                paramRes.Rs = paramTx.Rs
                paramRes.SpS_in  = paramADC.SpS_out
                paramRes.SpS_out = 2

                sigRx = resample(sigRx, paramRes)

                # CD compensation
                sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

                # Downsampling to 2 sps and re-synchronization with transmitted sequences
                sigRx = sigRx.reshape(-1, 1)

                symbRx = symbolSync(sigRx, symbTx, 2)

                # Power normalization
                x = sigRx
                d = symbRx

                x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
                d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

                # Adaptive equalization
                mod = QAMModem(m=paramTx.M)

                paramEq = parameters()
                paramEq.nTaps = 15
                paramEq.SpS = 2
                paramEq.mu = [1e-3, 5e-4]
                paramEq.numIter = 5
                paramEq.storeCoeff = False
                paramEq.alg = ["da-rde", "rde"]
                paramEq.M = paramTx.M
                paramEq.L = [20000, 80000]
                paramEq.prgsBar = False

                y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

                # Carrier phase recovery
                paramCPR = parameters()
                paramCPR.alg = "bps"
                paramCPR.M = paramTx.M
                paramCPR.N = 85
                paramCPR.B = 64
                paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

                y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

                y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

                # correct for (possible) phase ambiguity
                for k in range(y_CPR.shape[1]):
                    rot = np.mean(d[:, k] / y_CPR[:, k])
                    y_CPR[:, k] = rot * y_CPR[:, k]

                y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

                discard = int(paramEq.L[0]/2)

                ind = np.arange(discard, d.shape[0] - discard)
                BER[indAlg,indPower,indSNR], SER[indAlg,indPower,indSNR], SNR[indAlg,indPower,indSNR] = fastBERcalc(y_CPR[ind, :], d[ind, :], paramTx.M, 'qam')
                GMI[indAlg,indPower,indSNR], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')
                MI[indAlg,indPower,indSNR] = monteCarloMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')

                print("Results:")
                print("BER: %.2e" %(BER[indAlg,indPower,indSNR]))
                print("SNR: %.2f dB" %(SNR[indAlg,indPower,indSNR]))
                print('SIR = ', round(10*np.log10(SIR[indAlg,indPower,indSNR]), 2), ' dB')
                print("GMI: %.2f bits\n" %(GMI[indAlg,indPower,indSNR]))

        if saveData:      
            # save the simulation data
            np.save(path_data+'CSPR_BER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile)+str('_awgn_symbTx'), BER)
            np.save(path_data+'CSPR_SNR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile)+str('_awgn_symbTx'), SNR)
            np.save(path_data+'CSPR_SIR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile)+str('_awgn_symbTx'), SIR)
            np.save(path_data+'CSPR_SER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile)+str('_awgn_symbTx'), SER)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 6.00 dB
Results:
BER: 1.88e-01
SNR: 5.36 dB
SIR =  11.0  dB
GMI: 1.47 bits



  0%|          | 0/1 [00:00<?, ?it/s]

...

CSPR = 15.00 dB
12500/12500 [==============================] - 119s 10ms/step
Results:
BER: 3.68e-02
SNR: 11.30 dB
SIR =  14.22  dB
GMI: 2.89 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 15.00 dB
12500/12500 [==============================] - 122s 10ms/step
Results:
BER: 2.40e-02
SNR: 12.13 dB
SIR =  14.17  dB
GMI: 3.13 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 15.00 dB
12500/12500 [==============================] - 118s 9ms/step
Results:
BER: 1.38e-02
SNR: 12.94 dB
SIR =  14.2  dB
GMI: 3.39 bits



## Simulation of a single polarization optical signal transmission

In [3]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 400000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
# paramTx.alphaRRC = 0.5         # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 100     # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
FO = paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12       # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate
Ts = 1 / Fs

# photodiode parameters
paramPD = parameters()
paramPD.B  = 1.1*paramTx.Rs
paramPD.Fs = Fs

## Run all Roll-off variations

In [5]:
Rolloff = np.arange(0.05, 0.95, 0.05)
BER = np.zeros((3, len(Rolloff)))
SER = np.zeros((3, len(Rolloff)))
GMI = np.zeros((3, len(Rolloff)))
MI  = np.zeros((3, len(Rolloff)))
SNR = np.zeros((3, len(Rolloff)))
SIR = np.zeros((3, len(Rolloff)))

for indfile in range(0, Interactions):
    for indAlg, alg in enumerate(['KK', 'MLP', 'CONVNET']):
        for indRoll, rollOff in enumerate(tqdm(Rolloff)):
            
            paramTx.alphaRRC = rollOff 
            # generate optical signal signal
            sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

            # simulate linear signal propagation
            sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

            symbTx = symbTx_[:, :, chIndex]
            Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

            # generate LO field
            π = np.pi
            t = np.arange(0, len(sigCh))*Ts
            ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

            sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

            # Add LO to the received signal
            sigRx = np.sqrt(Plo) + sigCh* np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
            sfm   = sigRx.copy()

            print('CSPR = %.2f dB'%(10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

            # simulate ideal direct-detection optical receiver
            Ipd = photodiode(sigRx, paramPD)
            Amp = np.sqrt(Ipd.real)
            Amp = resample(Amp, paramADC).real

            # resampling to ADC sampling rate
            sigCh = resample(sigCh, paramADC)
            sfm = resample(sfm, paramADC)
            newFs = paramADC.SpS_out*paramTx.Rs

            sfm = sfm/np.sqrt(signal_power(sfm))
            if alg == 'KK':
                # Kramers-Kronig phase-retrieval
                phiTime = KramersKronigRx(Amp, newFs)
                # optical field reconstruction
                sigRx = Amp*np.exp(1j*phiTime)

            elif alg == 'MLP':
                # Mlp phase-retrieval
                model = tf.keras.models.load_model(path_mlp+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2))) 
                #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                # create set of input features
                X_input = np.zeros((len(sfm), N)) #(L,N)

                for indPhase in range(len(sfm)):
                    X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                sigRx_NN = model.predict(X_input)
                # optical field reconstruction
                sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]

            elif alg == 'CONVNET':
                # ConvNet phase-retrieval
                model = tf.keras.models.load_model(path_conv+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2)))
                #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                # create set of input features
                X_input = np.zeros((len(sfm), N)) #(L,N)

                for indPhase in range(len(sfm)):
                    X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                sigRx_NN = model.predict(X_input)
                # optical field reconstruction
                sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]    

            # remove DC level
            sigRx -= np.mean(sigRx) # np.sqrt(Plo)  

            # downshift to baseband
            t = np.arange(0, len(sigRx))*(1/newFs)
            sigRx *= np.exp(-1j * (2 * π * FO * t))
            
            # Matched filtering
            if paramTx.pulse == "nrz":
                pulse = pulseShape("nrz", paramADC.SpS_out)
            elif paramTx.pulse == "rrc":
                pulse = pulseShape(
                    "rrc", paramADC.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs
                )

            pulse = pulse / np.max(np.abs(pulse))
            sigRx = firFilter(pulse, sigRx)
            sigCh = firFilter(pulse, sigCh)
            
            # correct for (possible) phase ambiguity
            rot = np.mean(sigCh/sigRx)
            sigRx = rot * sigRx
            sigRx = sigRx / np.sqrt(signal_power(sigRx))

            intf = sigRx/np.sqrt(signal_power(sigRx))-sigCh/np.sqrt(signal_power(sigCh))

            SIR[indAlg, indRoll] = 1/signal_power(intf)


            # resample to 2 samples/symbol:
            paramRes = parameters()
            paramRes.Rs = paramTx.Rs
            paramRes.SpS_in  = paramADC.SpS_out
            paramRes.SpS_out = 2

            sigRx = resample(sigRx, paramRes)

            # CD compensation
            sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

            # Downsampling to 2 sps and re-synchronization with transmitted sequences
            sigRx = sigRx.reshape(-1, 1)

            symbRx = symbolSync(sigRx, symbTx, 2)

            # Power normalization
            x = sigRx
            d = symbRx

            x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
            d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

            # Adaptive equalization
            mod = QAMModem(m=paramTx.M)

            paramEq = parameters()
            paramEq.nTaps = 15
            paramEq.SpS = 2
            paramEq.mu = [1e-3, 5e-4]
            paramEq.numIter = 5
            paramEq.storeCoeff = False
            paramEq.alg = ["da-rde", "rde"]
            paramEq.M = paramTx.M
            paramEq.L = [20000, 80000]
            paramEq.prgsBar = False

            y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

            # Carrier phase recovery
            paramCPR = parameters()
            paramCPR.alg = "bps"
            paramCPR.M = paramTx.M
            paramCPR.N = 85
            paramCPR.B = 64
            paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

            y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            # correct for (possible) phase ambiguity
            for k in range(y_CPR.shape[1]):
                rot = np.mean(d[:, k] / y_CPR[:, k])
                y_CPR[:, k] = rot * y_CPR[:, k]

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            discard = int(paramEq.L[0]/2)

            ind = np.arange(discard, d.shape[0] - discard)
            BER[indAlg,indRoll], SER[indAlg,indRoll], SNR[indAlg,indRoll] = fastBERcalc(y_CPR[ind, :], d[ind, :], paramTx.M, 'qam')
            GMI[indAlg,indRoll], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')
            MI[indAlg,indRoll] = monteCarloMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')

            print("Results:")
            print("BER: %.2e" %(BER[indAlg,indRoll]))
            print("SNR: %.2f dB" %(SNR[indAlg,indRoll]))
            print('SIR = ', round(10*np.log10(SIR[indAlg,indRoll]), 2), ' dB')
            print("GMI: %.2f bits\n" %(GMI[indAlg,indRoll]))

    if saveData:
        # save the simulation data
        np.save(path_data+'Rolloff_BER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), BER)
        np.save(path_data+'Rolloff_SNR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SNR)
        np.save(path_data+'Rolloff_SIR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SIR)
        np.save(path_data+'Rolloff_SER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SER)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
Results:
BER: 0.00e+00
SNR: 24.03 dB
SIR =  22.93  dB
GMI: 4.00 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
Results:
BER: 0.00e+00
SNR: 22.35 dB
SIR =  19.78  dB
GMI: 4.00 bits



...

CSPR = 12.00 dB
12500/12500 [==============================] - 71s 6ms/step
Results:
BER: 1.87e-05
SNR: 20.00 dB
SIR =  12.68  dB
GMI: 4.00 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
12500/12500 [==============================] - 69s 5ms/step
Results:
BER: 4.06e-05
SNR: 19.70 dB
SIR =  12.23  dB
GMI: 4.00 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
12500/12500 [==============================] - 68s 5ms/step
Results:
BER: 3.75e-05
SNR: 19.68 dB
SIR =  11.91  dB
GMI: 4.00 bits



## Simulation of a single polarization optical signal transmission

In [10]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 400000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
paramTx.alphaRRC = 0.5         # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 100     # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
# FO = paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12       # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate
Ts = 1 / Fs

# photodiode parameters
paramPD = parameters()
paramPD.B  = 1.1*paramTx.Rs
paramPD.Fs = Fs

## Run all *Frequency offset* (FO) variations

In [12]:
FO_Values = np.arange(0e9, 6e9, 1e9)
BER = np.zeros((3, len(FO_Values)))
SER = np.zeros((3, len(FO_Values)))
GMI = np.zeros((3, len(FO_Values)))
MI  = np.zeros((3, len(FO_Values)))
SNR = np.zeros((3, len(FO_Values)))
SIR = np.zeros((3, len(FO_Values)))

for indfile in range(0, Interactions):
    for indAlg, alg in enumerate(['KK', 'MLP', 'CONVNET']):
        for indFO, FOfreq in enumerate(tqdm(FO_Values)):
            
            FO = paramTx.Rs/2 + FOfreq
            # generate optical signal signal
            sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

            # simulate linear signal propagation
            sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

            symbTx = symbTx_[:, :, chIndex]
            Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

            # generate LO field
            π = np.pi
            t = np.arange(0, len(sigCh))*Ts
            ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

            sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

            # Add LO to the received signal
            sigRx = np.sqrt(Plo) + sigCh* np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
            sfm   = sigRx.copy()

            print('CSPR = %.2f dB'%(10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

            # simulate ideal direct-detection optical receiver
            Ipd = photodiode(sigRx, paramPD)
            Amp = np.sqrt(Ipd.real)
            Amp = resample(Amp, paramADC).real

            # resampling to ADC sampling rate
            sigCh = resample(sigCh, paramADC)
            sfm = resample(sfm, paramADC)
            newFs = paramADC.SpS_out*paramTx.Rs

            sfm = sfm/np.sqrt(signal_power(sfm))
            if alg == 'KK':
                # Kramers-Kronig phase-retrieval
                phiTime = KramersKronigRx(Amp, newFs)
                # optical field reconstruction
                sigRx = Amp*np.exp(1j*phiTime)
                
            elif alg == 'MLP':
                # Mlp phase-retrieval
                model = tf.keras.models.load_model(path_mlp+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2))) 
                # #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                # create set of input features
                X_input = np.zeros((len(sfm), N)) #(L,N)

                for indPhase in range(len(sfm)):
                    X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                sigRx_NN = model.predict(X_input)
                # optical field reconstruction
                sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]

            elif alg == 'CONVNET':
                # ConvNet phase-retrieval
                model = tf.keras.models.load_model(path_conv+'testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2)))
                #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
                sigAmp  = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)
                # create set of input features
                X_input = np.zeros((len(sfm), N)) #(L,N)

                for indPhase in range(len(sfm)):
                    X_input[indPhase] = sigAmp[indPhase:N+indPhase]

                sigRx_NN = model.predict(X_input)
                # optical field reconstruction
                sigRx = sigRx_NN[:,0]+1j*sigRx_NN[:,1]    

            # remove DC level
            sigRx -= np.mean(sigRx) # np.sqrt(Plo)  

            # downshift to baseband
            t = np.arange(0, len(sigRx))*(1/newFs)
            sigRx *= np.exp(-1j * (2 * π * FO * t))
            
            # Matched filtering
            if paramTx.pulse == "nrz":
                pulse = pulseShape("nrz", paramADC.SpS_out)
            elif paramTx.pulse == "rrc":
                pulse = pulseShape(
                    "rrc", paramADC.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs
                )

            pulse = pulse / np.max(np.abs(pulse))
            sigRx = firFilter(pulse, sigRx)
            sigCh = firFilter(pulse, sigCh)
            
            # correct for (possible) phase ambiguity
            rot = np.mean(sigCh/sigRx)
            sigRx = rot * sigRx
            sigRx = sigRx / np.sqrt(signal_power(sigRx))

            intf = sigRx/np.sqrt(signal_power(sigRx))-sigCh/np.sqrt(signal_power(sigCh))

            SIR[indAlg, indFO] = 1/signal_power(intf)

            # resample to 2 samples/symbol:
            paramRes = parameters()
            paramRes.Rs = paramTx.Rs
            paramRes.SpS_in  = paramADC.SpS_out
            paramRes.SpS_out = 2

            sigRx = resample(sigRx, paramRes)

            # CD compensation
            sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

            # Downsampling to 2 sps and re-synchronization with transmitted sequences
            sigRx = sigRx.reshape(-1, 1)

            symbRx = symbolSync(sigRx, symbTx, 2)

            # Power normalization
            x = sigRx
            d = symbRx

            x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
            d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

            # Adaptive equalization
            mod = QAMModem(m=paramTx.M)

            paramEq = parameters()
            paramEq.nTaps = 15
            paramEq.SpS = 2
            paramEq.mu = [1e-3, 5e-4]
            paramEq.numIter = 5
            paramEq.storeCoeff = False
            paramEq.alg = ["da-rde", "rde"]
            paramEq.M = paramTx.M
            paramEq.L = [20000, 80000]
            paramEq.prgsBar = False

            y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

            # Carrier phase recovery
            paramCPR = parameters()
            paramCPR.alg = "bps"
            paramCPR.M = paramTx.M
            paramCPR.N = 85
            paramCPR.B = 64
            paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

            y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            # correct for (possible) phase ambiguity
            for k in range(y_CPR.shape[1]):
                rot = np.mean(d[:, k] / y_CPR[:, k])
                y_CPR[:, k] = rot * y_CPR[:, k]

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            discard = int(paramEq.L[0]/2)

            ind = np.arange(discard, d.shape[0] - discard)
            BER[indAlg,indFO], SER[indAlg,indFO], SNR[indAlg,indFO] = fastBERcalc(y_CPR[ind, :], d[ind, :], paramTx.M, 'qam')
            GMI[indAlg,indFO], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')
            MI[indAlg,indFO] = monteCarloMI(y_CPR[ind, :], d[ind, :],  paramTx.M, 'qam')

            print("Results:")
            print("BER: %.2e" %(BER[indAlg,indFO]))
            print("SNR: %.2f dB" %(SNR[indAlg,indFO]))
            print('SIR = ', round(10*np.log10(SIR[indAlg,indFO]), 2), ' dB')
            print("GMI: %.2f bits\n" %(GMI[indAlg,indFO]))

    if saveData:
        # save the simulation data
        np.save(path_data+'Conv_FO_BER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), BER)
        np.save(path_data+'Conv_FO_SNR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SNR)
        np.save(path_data+'Conv_FO_SIR_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SIR)
        np.save(path_data+'Conv_FO_SER_SpS_'+str(paramADC.SpS_out)+'_'+str(N)+'_Sample_'+str(indfile), SER)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

CSPR = 12.00 dB
12500/12500 [==============================] - 80s 6ms/step
Results:
BER: 0.00e+00
SNR: 28.76 dB
SIR =  22.27  dB
GMI: 4.00 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
12500/12500 [==============================] - 75s 6ms/step
Results:
BER: 0.00e+00
SNR: 30.72 dB
SIR =  26.05  dB
GMI: 4.00 bits



  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 12.00 dB
12500/12500 [==============================] - 77s 6ms/step
Results:
BER: 0.00e+00
SNR: 32.49 dB
SIR =  30.63  dB
GMI: 4.00 bits

